### Setup paths

In [1]:
import sys
import os

In [2]:
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

### Imports

In [3]:
from data_pull_and_prep.audio_from_yt import download_audio

### Download audio files from YouTube

In [4]:
# Usage
video_url = "https://www.youtube.com/watch?v=vcEVgN4eET8"  # Replace with your video URL
video_name = "test2"  # Replace with your video name
output_dir = project_root+"/data/testing/"  # Replace with your output directory

download_audio(video_url, video_name, output_dir)

Audio downloaded: /Users/rishikeshdhayarkar/rag-audio-indexing/data/testing/test2.mp3
